In [1]:
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import split, explode,to_date
import shutil
import gc

spark = SparkSession.builder \
    .appName("refine_gkg_counts") \
    .config("spark.driver.memory", "20g")\
    .getOrCreate()
    # .config("spark.memory.offHeap.enabled",True)\
    # .config("spark.executor.memory", "10g")\
    # .config("spark.memory.offHeap.size","8g") \

base_dir = '/home/oscar/budasbi-repos/factored-datathon-2024-voyager/bucket_contents/raw/gkg_counts'
parquet_path = '/home/oscar/budasbi-repos/factored-datathon-2024-voyager/parquet/'
if os.path.exists(parquet_path):
    shutil.rmtree(parquet_path)
    os.makedirs(parquet_path)


24/08/23 20:39:52 WARN Utils: Your hostname, ORION resolves to a loopback address: 127.0.1.1; using 192.168.10.14 instead (on interface wlp0s20f3)
24/08/23 20:39:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/23 20:39:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/23 20:39:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
spark.sparkContext

<SparkContext master=local[*] appName=refine_gkg_counts>

In [17]:
def cleaned_to_parquet(csv_filepath):
    gkg_counts = spark.read.csv(csv_filepath, header=True, sep='\t',inferSchema=True)
    gkg_colums=['date','numarts', 'count_type', 'number', 'object_type', 'geo_type', 'geo_fullname', 'geo_country_code', 'geo_adm1_code', 'geo_lat', 'geo_long','geo_feature_id', 'cameo_event_ids', 'sources', 'source_urls' ]
    gkg_counts_columns = gkg_counts.toDF(*gkg_colums)
    gkg_counts_date = gkg_counts_columns.withColumn('date', to_date('date', 'yyyyMMdd'))
    gkg_counts_date.show(truncate=False)
    #Split Cameo_event_ids
    gkg_counts_split = gkg_counts_date.withColumn("cameo_event_ids_split", split(gkg_counts_date["cameo_event_ids"], ","))
    gkg_counts_explode = gkg_counts_split.withColumn("cameo_event_ids", explode(gkg_counts_split["cameo_event_ids_split"]))
    gkg_counts_ex = gkg_counts_explode.drop("cameo_event_ids_split")
    print(csv_filepath)
    gkg_counts_nodups = gkg_counts_ex.dropDuplicates()
    
    
    gkg_counts.unpersist()
    gkg_counts_columns.unpersist()
    gkg_counts_date.unpersist()
    gkg_counts_split.unpersist()
    gkg_counts_explode.unpersist()
    gkg_counts_ex.unpersist()
    
    gkg_counts_nodups.write\
    .mode('append')\
    .format('parquet')\
    .option('compression', 'snappy')\
    .partitionBy('date')\
    .save(parquet_path)
    return gkg_counts_nodups 
    gkg_counts_nodups.unpersist()
    gc.collect()

In [18]:
gkg_files = []
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".csv"):
            gkg_files.append(os.path.join(root, file))


In [19]:
if gkg_files:
    for file in gkg_files:
        test = cleaned_to_parquet(file)
        break
    # gkg_counts = spark.read.csv(gkg_files, header=True, sep='\t',inferSchema=True)
else:   
    print("We didn't find any CSV file")


+----------+-------+------------------------------+------+-------------------+--------+------------------------------------------------------------+----------------+-------------+-------+--------+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
test.printSchema()

root
 |-- date: date (nullable = true)
 |-- numarts: integer (nullable = true)
 |-- count_type: string (nullable = true)
 |-- number: long (nullable = true)
 |-- object_type: string (nullable = true)
 |-- geo_type: integer (nullable = true)
 |-- geo_fullname: string (nullable = true)
 |-- geo_country_code: string (nullable = true)
 |-- geo_adm1_code: string (nullable = true)
 |-- geo_lat: double (nullable = true)
 |-- geo_long: double (nullable = true)
 |-- geo_feature_id: string (nullable = true)
 |-- cameo_event_ids: string (nullable = false)
 |-- sources: string (nullable = true)
 |-- source_urls: string (nullable = true)

